In [1]:
!pip install --upgrade paddlenlp==2.0.0b4 -i https://pypi.org/simple

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.3/164.3 kB 104.4 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: paddlenlp
    Found existing installation: paddlenlp 2.0.7
    Uninstalling paddlenlp-2.0.7:
      Successfully uninstalled paddlenlp-2.0.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
paddlehub 2.0.4 requires paddlenlp>=2.0.0rc5, but you have paddlenlp 2.0.0b4 which is incompatible.

[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [9]:
import re
import jieba
import os 
import random
import paddle
import paddlenlp as ppnlp
from paddlenlp.data import Stack, Pad, Tuple
import paddle.nn.functional as F
import paddle.nn as nn
from visualdl import LogWriter
import numpy as np
from functools import partial #partial()函数可以用来固定某些参数值，并返回一个新的callable对象
import os, zipfile
import io, random, json
print(paddle.__version__)

2.0.2


In [7]:
#解压原始数据集，将Rumor_Dataset.zip解压至data目录下
src_path="data/data20519/Rumor_Dataset.zip"
target_path="/home/aistudio/data/Chinese_Rumor_Dataset-master"
if(not os.path.isdir(target_path)):
    z = zipfile.ZipFile(src_path, 'r')
    z.extractall(path=target_path)
    z.close()

In [10]:
#数据文件路径
rumor_class_dirs = os.listdir(target_path+"/Chinese_Rumor_Dataset-master/CED_Dataset/rumor-repost/")

non_rumor_class_dirs = os.listdir(target_path+"/Chinese_Rumor_Dataset-master/CED_Dataset/non-rumor-repost/")
original_microblog = target_path+"/Chinese_Rumor_Dataset-master/CED_Dataset/original-microblog/"

rumor_label="0"
non_rumor_label="1"


rumor_num = 0
non_rumor_num = 0
all_rumor_list = []
all_non_rumor_list = []

#解析数据
for rumor_class_dir in rumor_class_dirs: 
    if(rumor_class_dir != '.DS_Store'):
        #遍历数据，并解析
        with open(original_microblog + rumor_class_dir, 'r') as f:
	        rumor_content = f.read()
        rumor_dict = json.loads(rumor_content)
        all_rumor_list.append(rumor_label+"\t"+rumor_dict["text"]+"\n")
        rumor_num +=1

#解析数据
for non_rumor_class_dir in non_rumor_class_dirs: 
    if(non_rumor_class_dir != '.DS_Store'):
        with open(original_microblog + non_rumor_class_dir, 'r') as f2:
	        non_rumor_content = f2.read()
        non_rumor_dict = json.loads(non_rumor_content)
        all_non_rumor_list.append(non_rumor_label+"\t"+non_rumor_dict["text"]+"\n")
        non_rumor_num +=1

In [11]:
data_list_path="/home/aistudio/data/"
all_data_path=data_list_path + "all_data.txt"

all_data_list = all_rumor_list + all_non_rumor_list

random.shuffle(all_data_list)

#在生成all_data.txt之前，首先将其清空
with open(all_data_path, 'w') as f:
    f.seek(0)
    f.truncate() 
    
with open(all_data_path, 'a') as f:
    for data in all_data_list:
        f.write(data) 

In [12]:
# 生成数据字典
def create_dict(data_path, dict_path):
    with open(dict_path, 'w') as f:
        f.seek(0)
        f.truncate() 

    dict_set = set()
    # 读取全部数据
    with open(data_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    # 把数据生成一个元组
    for line in lines:
        content = line.split('\t')[-1].replace('\n', '')
        for s in content:
            dict_set.add(s)
    # 把元组转换成字典，一个字对应一个数字
    dict_list = []
    i = 0
    for s in dict_set:
        dict_list.append([s, i])
        i += 1
    # 添加未知字符
    dict_txt = dict(dict_list)
    end_dict = {"<unk>": i}
    dict_txt.update(end_dict)
    end_dict = {"<pad>": i+1}
    dict_txt.update(end_dict)
    # 把这些字典保存到本地中
    with open(dict_path, 'w', encoding='utf-8') as f:
        f.write(str(dict_txt))
        
    print("数据字典生成完成！")

In [13]:
def load_vocab(file_path):
    fr = open(file_path, 'r', encoding='utf8')
    vocab = eval(fr.read())   #读取的str转换为字典
    fr.close()

    return vocab

def f_write_txt(words, dict_txt, label):
    labs = ""
    for s in words:
        lab = str(dict_txt[s])
        labs = labs + lab + ','
    labs = labs[:-1]
    labs = labs + '\t' + label + '\n'
    return labs

def create_data_list(data_path, train_path, test_path, eval_path, dict_path):
    
    dict_txt = load_vocab(dict_path)
    with open(data_path, 'r', encoding='utf-8') as f_data:
        lines = f_data.readlines()

    i = 0
    maxlen = 0
    with open(test_path, 'a', encoding='utf-8') as f_test,open(train_path, 'a', encoding='utf-8') as f_train, open(eval_path, 'a', encoding='utf-8') as f_eval:
        for line in lines:
            words = line.split('\t')[-1].replace('\n', '')
            maxlen = max(maxlen, len(words))
            label = line.split('\t')[0]
            labs = f_write_txt(words, dict_txt, label)
            # 每8个 抽取一个数据用于验证
            if i % 8 == 0:
                f_test.write(labs)
            if i % 5 == 0:
                f_eval.write(labs)
            else:
                f_train.write(labs)
            i += 1
    print("数据列表生成完成！")
    print(maxlen)


In [14]:
data_root_path = "/home/aistudio/data/" 
data_path = os.path.join(data_root_path, 'all_data.txt')
train_path = os.path.join(data_list_path, 'train_list.txt')
test_path = os.path.join(data_list_path, 'test_list.txt')
eval_path = os.path.join(data_list_path, 'eval_list.txt')
dict_path = os.path.join(data_root_path, "dict.txt")

# 创建数据字典
create_dict(data_path, dict_path)

# 创建数据列表

#在生成数据之前，首先将eval_list.txt和train_list.txt清空
with open(test_path, 'w', encoding='utf-8') as f_test:
    f_test.seek(0)
    f_test.truncate()
with open(train_path, 'w', encoding='utf-8') as f_train:
    f_train.seek(0)
    f_train.truncate() 
with open(eval_path, 'w', encoding='utf-8') as f_eval:
    f_eval.seek(0)
    f_eval.truncate() 

create_data_list(data_path, train_path, test_path ,eval_path, dict_path)


数据字典生成完成！
数据列表生成完成！
226


In [16]:
# 打印前2条训练数据
vocab = load_vocab(dict_path)

def ids_to_str(ids):
    words = []
    for k in ids:
        w = list(vocab.keys())[list(vocab.values()).index(int(k))]
        words.append(w if isinstance(w, str) else w.decode('ASCII'))
    return " ".join(words)

In [17]:
f=open("eval_list.txt","w")

with io.open(eval_path, "r", encoding='utf8') as fin:
        i = 0
        for line in fin:
            i += 1
            cols = line.strip().split("\t")
            if len(cols) != 2:
                sys.stderr.write("[NOTICE] Error Format Line!")
                continue
            label = int(cols[1])
            wids = cols[0].split(",")
            strline = ids_to_str(wids) + "\t" + str(label) + "\n"
            f.writelines(strline)

In [18]:
f=open("train.txt","w")

with io.open(train_path, "r", encoding='utf8') as fin:
        i = 0
        for line in fin:
            i += 1
            cols = line.strip().split("\t")
            if len(cols) != 2:
                sys.stderr.write("[NOTICE] Error Format Line!")
                continue
            label = int(cols[1])
            wids = cols[0].split(",")
            strline = ids_to_str(wids) + "\t" + str(label) + "\n"
            f.writelines(strline)

In [19]:
f=open("test_list.txt","w")

with io.open(test_path, "r", encoding='utf8') as fin:
        i = 0
        for line in fin:
            i += 1
            cols = line.strip().split("\t")
            if len(cols) != 2:
                sys.stderr.write("[NOTICE] Error Format Line!")
                continue
            label = int(cols[1])
            wids = cols[0].split(",")
            strline = ids_to_str(wids) + "\t" + str(label) + "\n"
            f.writelines(strline)

In [30]:
#数据处理已经做好可以从这里开始运行代码
class SelfDefinedDataset(paddle.io.Dataset):
    def __init__(self, data):
        super(SelfDefinedDataset, self).__init__()
        self.data = data

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return len(self.data)
        
    def get_labels(self):
        return ["0", "1"]

def txt_to_list(file_name):
    res_list = []
    for line in open(file_name):
        res_list.append(line.strip().split('\t'))
    return res_list

trainlst = txt_to_list('train.txt')
devlst = txt_to_list('eval_list.txt')
testlst = txt_to_list('test_list.txt')

train_ds, dev_ds, test_ds = SelfDefinedDataset.get_datasets([trainlst, devlst, testlst])

In [34]:
label_list = train_ds.get_labels()
#看看数据长什么样子，分别打印训练集、验证集、测试集的前3条数据。
print("训练集数据：{}\n".format(train_ds[0:3]))
print("验证集数据:{}\n".format(dev_ds[0:3]))
print("测试集数据:{}\n".format(test_ds[0:3]))

print("训练集样本个数:{}".format(len(train_ds)))
print("验证集样本个数:{}".format(len(dev_ds)))
print("测试集样本个数:{}".format(len(test_ds)))

训练集数据：[['有 人 说 ， 失 眠 ， 是 因 为 你 正 在 别 人 的 梦 里 辛 苦 地 忙 碌 着 。     晚 安', '1'], ['# 自 然 堂 全 国 路 演 随 手 拍 # 自 然 堂 舞 林 争 霸 冠 军 果 然 是 我 喜 欢 的 @ 张 傲 月   ， 一 路 走 来 一 直 都 非 常 支 持 他 ， 没 想 到 六 一 儿 童 节 还 在 王 府 井 东 安 市 场 看 到 他 参 加 的 自 然 堂 路 演 活 动 ， 在 现 场 看 见 了 他 真 人 跳 舞 哦 ， 喜 欢 张 傲 月 的 女 粉 们 ， 赶 紧 来 收 图 啦 ！ 在 现 场 我 还 体 验 了 自 然 堂 的 产 品 ， 更 加 期 待 惊 喜 大 奖 了 ~', '1'], ['[ 转 ]   有 车 的 朋 友 要 注 意 了 啊 ！ ！ ！ 加 油 时 你 要 注 意 加 油 员 的 这 个 小 动 作 了 . . .', '0']]

验证集数据:[['【 拾 金 不 昧 要 慎 重 】 长 沙 一 小 孩 上 学 路 上 捡 到 3 万 元 ， 就 坐 在 那 里 等 失 主 ， 结 果 被 人 冒 领 ， 小 孩 回 校 告 知 老 师 后 得 到 表 扬 ， 不 久 真 失 主 听 说 小 孩 拾 金 不 昧 的 事 后 找 上 门 索 钱 ， 未 得 ， 就 告 上 法 庭 ， 最 后 判 小 孩 家 长 赔 2 0 % 即 6 0 0 0 元 。 这 是 真 实 事 件 ， 请 大 家 以 此 为 鉴 ， 教 育 小 孩 拾 金 不 昧 要 慎 重 啊 。 （ 转 ）', '0'], ['1 3 1 2 3 1   # 东 方 卫 视 跨 年 倒 计 时 #   # 吴 亦 凡 #   # k r i s #   天 生 丽 质 难 自 弃 、 含 情 凝 睇 谢 君 王 、 回 眸 一 笑 百 媚 生 、 此 恨 绵 绵 无 绝 期   [ 挖 鼻 屎 ]   长 恨 歌 那 么 长 当 年 是 怎 么 背 下 来 的 [ 偷 笑 ]   第 一 张 图 我 打 赌 你 无 法 盯 着 超 过 1 0 秒 . . . . . . . 这 张 是 个 宝 ， 谁 偷 我 跟 谁 吵 ！', '1'], ['2 0 1 3 ， 我 们

In [33]:
#验证清洗脏数据
for line in train_ds:
    if len(line) != 2:
        print("error")
        print(line)
        break

In [44]:
tokenizer = ppnlp.transformers.BertTokenizer.from_pretrained("bert-base-chinese")

#数据预处理
def convert_example(example,tokenizer,label_list,max_seq_length=256,is_test=False):
    if is_test:
        text = example
    else:
        text, label = example
    #tokenizer.encode方法能够完成切分token，映射token ID以及拼接特殊token
    encoded_inputs = tokenizer.encode(text=text, max_seq_len=max_seq_length)
    input_ids = encoded_inputs["input_ids"]
    #注意，在早前的PaddleNLP版本中，token_type_ids叫做segment_ids
    segment_ids = encoded_inputs["segment_ids"]

    if not is_test:
        label_map = {}
        for (i, l) in enumerate(label_list):
            label_map[l] = i

        label = label_map[label]
        label = np.array([label], dtype="int64")
        return input_ids, segment_ids, label
    else:
        return input_ids, segment_ids

#数据迭代器构造方法
def create_dataloader(dataset, trans_fn=None, mode='train', batch_size=1, use_gpu=False, pad_token_id=0, batchify_fn=None):
    if trans_fn:
        dataset = dataset.apply(trans_fn, lazy=True)

    if mode == 'train' and use_gpu:
        sampler = paddle.io.DistributedBatchSampler(dataset=dataset, batch_size=batch_size, shuffle=True)
    else:
        shuffle = True if mode == 'train' else False #如果不是训练集，则不打乱顺序
        sampler = paddle.io.BatchSampler(dataset=dataset, batch_size=batch_size, shuffle=shuffle) #生成一个取样器
    dataloader = paddle.io.DataLoader(dataset, batch_sampler=sampler, return_list=True, collate_fn=batchify_fn)
    return dataloader

#使用partial()来固定convert_example函数的tokenizer, label_list, max_seq_length, is_test等参数值
trans_fn = partial(convert_example, tokenizer=tokenizer, label_list=label_list, max_seq_length=128, is_test=False)
batchify_fn = lambda samples, fn=Tuple(Pad(axis=0,pad_val=tokenizer.pad_token_id), Pad(axis=0, pad_val=tokenizer.pad_token_id), Stack(dtype="int64")):[data for data in fn(samples)]
#训练集迭代器
train_loader = create_dataloader(train_ds, mode='train', batch_size=32, batchify_fn=batchify_fn, trans_fn=trans_fn)
#验证集迭代器
dev_loader = create_dataloader(dev_ds, mode='dev', batch_size=1, batchify_fn=batchify_fn, trans_fn=trans_fn)
#测试集迭代器
test_loader = create_dataloader(test_ds, mode='test', batch_size=32, batchify_fn=batchify_fn, trans_fn=trans_fn)

[2023-01-10 16:22:55,303] [    INFO] - Found /home/aistudio/.paddlenlp/models/bert-base-chinese/bert-base-chinese-vocab.txt


In [36]:
model = ppnlp.transformers.BertForSequenceClassification.from_pretrained("bert-base-chinese", num_classes=2)

[2023-01-10 16:17:56,785] [    INFO] - Downloading http://paddlenlp.bj.bcebos.com/models/transformers/bert/bert-base-chinese.pdparams and saved to /home/aistudio/.paddlenlp/models/bert-base-chinese
[2023-01-10 16:17:56,790] [    INFO] - Downloading bert-base-chinese.pdparams from http://paddlenlp.bj.bcebos.com/models/transformers/bert/bert-base-chinese.pdparams
100%|██████████| 696494/696494 [01:27<00:00, 8004.19it/s] 
W0110 16:19:23.928725   270 device_context.cc:362] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 10.1
W0110 16:19:23.935091   270 device_context.cc:372] device: 0, cuDNN Version: 7.6.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1303: UserWarning: Skip loading for classifier.weight. classifier.weight is not found in the provided dict.
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/

In [37]:
#设置训练超参数

#学习率
learning_rate = 1e-5 
#训练轮次
epochs = 3
#学习率预热比率
warmup_proption = 0.1
#权重衰减系数
weight_decay = 0.01

num_training_steps = len(train_loader) * epochs
num_warmup_steps = int(warmup_proption * num_training_steps)

def get_lr_factor(current_step):
    if current_step < num_warmup_steps:
        return float(current_step) / float(max(1, num_warmup_steps))
    else:
        return max(0.0,
                    float(num_training_steps - current_step) /
                    float(max(1, num_training_steps - num_warmup_steps)))
#学习率调度器
lr_scheduler = paddle.optimizer.lr.LambdaDecay(learning_rate, lr_lambda=lambda current_step: get_lr_factor(current_step))

#优化器
optimizer = paddle.optimizer.AdamW(
    learning_rate=lr_scheduler,
    parameters=model.parameters(),
    weight_decay=weight_decay,
    apply_decay_param_fun=lambda x: x in [
        p.name for n, p in model.named_parameters()
        if not any(nd in n for nd in ["bias", "norm"])
    ])

#损失函数
criterion = paddle.nn.loss.CrossEntropyLoss()
#评估函数
metric = paddle.metric.Accuracy()

In [38]:
def evaluate(model, criterion, metric, data_loader):
    model.eval()
    metric.reset()
    losses = []
    for batch in data_loader:
        input_ids, segment_ids, labels = batch
        logits = model(input_ids, segment_ids)
        loss = criterion(logits, labels)
        losses.append(loss.numpy())
        correct = metric.compute(logits, labels)
        metric.update(correct)
        accu = metric.accumulate()
    print("eval loss: %.5f, accu: %.5f" % (np.mean(losses), accu))
    model.train()
    metric.reset()
    return np.mean(losses), accu

In [39]:
global_step = 0
with LogWriter(logdir="./log") as writer:
    for epoch in range(1, epochs + 1):    
        for step, batch in enumerate(train_loader, start=1): #从训练数据迭代器中取数据
            input_ids, segment_ids, labels = batch
            #print(segment_ids)
            logits = model(input_ids, segment_ids)
            loss = criterion(logits, labels) #计算损失
            probs = F.softmax(logits, axis=1)
            correct = metric.compute(probs, labels)
            metric.update(correct)
            acc = metric.accumulate()
            # print(acc)

            global_step += 1
            if global_step % 50 == 0 :
                print("global step %d, epoch: %d, batch: %d, loss: %.5f, acc: %.5f" % (global_step, epoch, step, loss, acc))
                #记录训练过程
                writer.add_scalar(tag="train/loss", step=global_step, value=loss)
                writer.add_scalar(tag="train/acc", step=global_step, value=acc)
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.clear_gradients()
        eval_loss, eval_acc = evaluate(model, criterion, metric, dev_loader)
        #记录评估过程
        writer.add_scalar(tag="eval/loss", step=epoch, value=eval_loss)
        writer.add_scalar(tag="eval/acc", step=epoch, value=eval_acc)

global step 50, epoch: 1, batch: 50, loss: 0.30604, acc: 0.68188
eval loss: 0.29938, accu: 0.88791
global step 100, epoch: 2, batch: 15, loss: 0.33418, acc: 0.89167
global step 150, epoch: 2, batch: 65, loss: 0.18857, acc: 0.91154
eval loss: 0.25920, accu: 0.90265
global step 200, epoch: 3, batch: 30, loss: 0.13239, acc: 0.94792
global step 250, epoch: 3, batch: 80, loss: 0.06305, acc: 0.95273
eval loss: 0.25578, accu: 0.91150


In [40]:
def predict(model, data, tokenizer, label_map, batch_size=1):
    examples = []
    results = []
    lable = []
    model.eval()
    for batch in data:
        input_ids, segment_ids, labels = batch
        input_ids = paddle.to_tensor(input_ids)
        segment_ids = paddle.to_tensor(segment_ids)
        logits = model(input_ids, segment_ids)
        probs = F.softmax(logits, axis=1)
        idx = paddle.argmax(probs, axis=1).numpy()
        idx = idx.tolist()
        # print(labels.numpy().tolist())
        # labels = [label_map[i] for i in idx]
        results.extend(idx)
        lable.extend(labels.numpy().tolist()[0])
    return results,lable

In [45]:
label_map = {0: '谣言', 1: '不是谣言'}
predictions,r = predict(model, dev_loader, tokenizer, label_map, batch_size=1)

from sklearn.metrics import classification_report
target_names = ["0","1"]
CR=classification_report(r, predictions, target_names=target_names,digits=4)
print(CR)

              precision    recall  f1-score   support

           0     0.9048    0.9048    0.9048       315
           1     0.9174    0.9174    0.9174       363

    accuracy                         0.9115       678
   macro avg     0.9111    0.9111    0.9111       678
weighted avg     0.9115    0.9115    0.9115       678

